### Tomo el archivo correspondiente a la campaña 2017-2018 en la correspondiente carpeta, veo el contenido para entender un poco mejor la estructura del mismo

In [1]:
import pandas as pd

df_liga = pd.read_csv('2017-18/1-liga.csv')

df_liga.head(5)

,Round,Date,Team 1,FT,HT,Team 2
0,?,(Fri) 18 Aug 2017 (33),CD Leganés (1),1-0,1-0,Deportivo Alavés (1)
1,?,(Fri) 18 Aug 2017 (33),Valencia CF (1),1-0,1-0,UD Las Palmas (1)
2,?,(Sat) 19 Aug 2017 (33),RC Celta Vigo (1),2-3,1-1,Real Sociedad (1)
3,?,(Sat) 19 Aug 2017 (33),Girona FC (1),2-2,2-0,Atlético Madrid (1)
4,?,(Sat) 19 Aug 2017 (33),Sevilla FC (1),1-1,1-1,RCD Español (1)


### Para un desarrollo más cómodo y limpio, voy a renombrar las columnas

In [2]:
df_liga.columns = ['Jornada', 'Fecha', 'Local', 'Resultado_Final', 'Resultado_Parcial', 'Visita']

df_liga.head(5)

,Jornada,Fecha,Local,Resultado_Final,Resultado_Parcial,Visita
0,?,(Fri) 18 Aug 2017 (33),CD Leganés (1),1-0,1-0,Deportivo Alavés (1)
1,?,(Fri) 18 Aug 2017 (33),Valencia CF (1),1-0,1-0,UD Las Palmas (1)
2,?,(Sat) 19 Aug 2017 (33),RC Celta Vigo (1),2-3,1-1,Real Sociedad (1)
3,?,(Sat) 19 Aug 2017 (33),Girona FC (1),2-2,2-0,Atlético Madrid (1)
4,?,(Sat) 19 Aug 2017 (33),Sevilla FC (1),1-1,1-1,RCD Español (1)


### Son todas columnas del tipo string, por ende debo parsear los campos "Local" y "Visita" para obtener las jornadas y emprolijar los nombres de los equipos

In [3]:
for index, row in df_liga.iterrows():
    local = row['Local']
    visita = row['Visita']
    
    jornada = int(local[local.find('(') + 1:local.find(')')])
    local_limpio = local[:local.find('(')].strip()
    visita_limpio = visita[:visita.find('(')].strip()
    
    df_liga.at[index, 'Jornada'] = jornada
    df_liga.at[index, 'Local'] = local_limpio
    df_liga.at[index, 'Visita'] = visita_limpio

df_liga.head(5)

,Jornada,Fecha,Local,Resultado_Final,Resultado_Parcial,Visita
0,1,(Fri) 18 Aug 2017 (33),CD Leganés,1-0,1-0,Deportivo Alavés
1,1,(Fri) 18 Aug 2017 (33),Valencia CF,1-0,1-0,UD Las Palmas
2,1,(Sat) 19 Aug 2017 (33),RC Celta Vigo,2-3,1-1,Real Sociedad
3,1,(Sat) 19 Aug 2017 (33),Girona FC,2-2,2-0,Atlético Madrid
4,1,(Sat) 19 Aug 2017 (33),Sevilla FC,1-1,1-1,RCD Español


### Con el dataset corregido, agrego 4 columnas nuevas: "Local_Puntos", "Visita_Puntos", "Local_Goles" y "Visita_Goles" donde calculo según corresponda la cantidad de puntos obtenidos por cada partido de cada jornada y los goles correspondientes a cada equipo

In [4]:
for index, row in df_liga.iterrows():
    resultado = row['Resultado_Final']
    local_puntos = 0
    visita_puntos = 0
    
    local_goles = int(resultado[:resultado.find('-')])
    visita_goles = int(resultado[resultado.find('-') + 1:])
    
    if local_goles > visita_goles:
        local_puntos = 3
    else:
        if visita_goles > local_goles:
            visita_puntos = 3
        else:
            local_puntos = 1
            visita_puntos = 1

    df_liga.at[index, 'Local_Puntos'] = local_puntos
    df_liga.at[index, 'Visita_Puntos'] = visita_puntos
    df_liga.at[index, 'Local_Goles'] = local_goles
    df_liga.at[index, 'Visita_Goles'] = visita_goles

df_liga['Local_Puntos'] = df_liga['Local_Puntos'].astype(int)
df_liga['Visita_Puntos'] = df_liga['Visita_Puntos'].astype(int)
df_liga['Local_Goles'] = df_liga['Local_Goles'].astype(int)
df_liga['Visita_Goles'] = df_liga['Visita_Goles'].astype(int)
df_liga.head(5)

,Jornada,Fecha,Local,Resultado_Final,Resultado_Parcial,Visita,Local_Puntos,Visita_Puntos,Local_Goles,Visita_Goles
0,1,(Fri) 18 Aug 2017 (33),CD Leganés,1-0,1-0,Deportivo Alavés,3,0,1,0
1,1,(Fri) 18 Aug 2017 (33),Valencia CF,1-0,1-0,UD Las Palmas,3,0,1,0
2,1,(Sat) 19 Aug 2017 (33),RC Celta Vigo,2-3,1-1,Real Sociedad,0,3,2,3
3,1,(Sat) 19 Aug 2017 (33),Girona FC,2-2,2-0,Atlético Madrid,1,1,2,2
4,1,(Sat) 19 Aug 2017 (33),Sevilla FC,1-1,1-1,RCD Español,1,1,1,1


### Con los puntos calculados, genero un nuevo DataFrame con los datos que quiero mostrar en la tabla: "Posicion", "Equipo", "Puntos", "Ganados", "Empatados", "Perdidos", "Goles_A_Favor", "Goles_En_Contra", "Diferencia_De_Gol"

In [5]:
df_equipos = df_liga.groupby('Local').count().reset_index()[['Local']]
df_equipos.columns = ['Equipo']

for index, row in df_equipos.iterrows():
    equipo = row['Equipo']
    puntos = 0
    goles_favor = 0
    goles_contra = 0
    ganados = 0
    empatados = 0
    perdidos = 0
    
    df_locales = df_liga[df_liga['Local'] == equipo]
    df_visitas = df_liga[df_liga['Visita'] == equipo]
    
    for _, row in df_locales.iterrows():
        local_puntos = row['Local_Puntos']
        local_goles = row['Local_Goles']
        visita_goles = row['Visita_Goles']
        
        puntos += local_puntos
        goles_favor += local_goles
        goles_contra += visita_goles
        
        if local_puntos == 3:
            ganados += 1
        else:
            if local_puntos == 1:
                empatados += 1
            else:
                perdidos += 1

    for _, row in df_visitas.iterrows():
        visita_puntos = row['Visita_Puntos']
        visita_goles = row['Visita_Goles']
        local_goles = row['Local_Goles']
        
        puntos += visita_puntos
        goles_favor += visita_goles
        goles_contra += local_goles
        
        if visita_puntos == 3:
            ganados += 1
        else:
            if visita_puntos == 1:
                empatados += 1
            else:
                perdidos += 1

    df_equipos.at[index, 'Puntos'] = puntos
    df_equipos.at[index, 'Ganados'] = ganados
    df_equipos.at[index, 'Empatados'] = empatados
    df_equipos.at[index, 'Perdidos'] = perdidos
    df_equipos.at[index, 'Goles_A_Favor'] = goles_favor
    df_equipos.at[index, 'Goles_En_Contra'] = goles_contra
    df_equipos.at[index, 'Diferencia_De_Gol'] = goles_favor - goles_contra

df_equipos['Puntos'] = df_equipos['Puntos'].astype(int)
df_equipos['Ganados'] = df_equipos['Ganados'].astype(int)
df_equipos['Empatados'] = df_equipos['Empatados'].astype(int)
df_equipos['Perdidos'] = df_equipos['Perdidos'].astype(int)
df_equipos['Goles_A_Favor'] = df_equipos['Goles_A_Favor'].astype(int)
df_equipos['Goles_En_Contra'] = df_equipos['Goles_En_Contra'].astype(int)
df_equipos['Diferencia_De_Gol'] = df_equipos['Diferencia_De_Gol'].astype(int)
df_equipos.head(5)

,Equipo,Puntos,Ganados,Empatados,Perdidos,Goles_A_Favor,Goles_En_Contra,Diferencia_De_Gol
0,Athletic Club Bilbao,43,10,13,15,41,49,-8
1,Atlético Madrid,79,23,10,5,58,22,36
2,CD Leganés,43,12,7,19,34,51,-17
3,Deportivo Alavés,47,15,2,21,40,50,-10
4,FC Barcelona,93,28,9,1,99,29,70


### La tabla final es simplemente el dataset anterior mostrado de forma ordenada por "Puntos" descendiente:

In [6]:
df_equipos = df_equipos.sort_values('Puntos', ascending = False).reset_index()
df_equipos['Posicion'] = df_equipos.index + 1
df_equipos = df_equipos[['Posicion', 'Equipo', 'Puntos', 'Ganados', 'Empatados', 'Perdidos', 'Goles_A_Favor', 'Goles_En_Contra', 'Diferencia_De_Gol']]
df_equipos.head(20)

,Posicion,Equipo,Puntos,Ganados,Empatados,Perdidos,Goles_A_Favor,Goles_En_Contra,Diferencia_De_Gol
0,1,FC Barcelona,93,28,9,1,99,29,70
1,2,Atlético Madrid,79,23,10,5,58,22,36
2,3,Real Madrid CF,76,22,10,6,94,44,50
3,4,Valencia CF,73,22,7,9,65,38,27
4,5,Villarreal CF,61,18,7,13,57,50,7
5,6,Real Betis,60,18,6,14,60,61,-1
6,7,Sevilla FC,58,17,7,14,49,58,-9
7,8,Getafe CF,55,15,10,13,42,33,9
8,9,Girona FC,51,14,9,15,50,59,-9
9,10,SD Eibar,51,14,9,15,44,50,-6


### Cargo la tabla para determinar la mejor inversión posible. Veo las primeras 5 filas para entender un poco mejor la estructura:

In [7]:
df_investment = pd.read_csv('investments.csv', encoding = 'ANSI')

df_investment.head(5)

,Team,Investment required,Profit per goal,Unnamed: 3
0,CD Leganés,"$120,000.00","$3,000.00",NaN
1,Valencia CF,"$420,000.00","$7,000.00",NaN
2,RC Celta Vigo,"$230,000.00","$6,000.00",NaN
3,Girona FC,"$100,000.00","$4,000.00",NaN
4,Sevilla FC,"$450,000.00","$10,000.00",NaN


### Limpio la última columna que no me sirve y redefino las columnas a "Equipo", "Inversion_Necesaria", "Ganancia_Por_Gol"

In [8]:
df_investment.columns = ['Equipo', 'Inversion_Necesaria', 'Ganancia_Por_Gol', 'unnamed']

df_investment = df_investment[['Equipo', 'Inversion_Necesaria', 'Ganancia_Por_Gol']]

df_investment.head(5)

,Equipo,Inversion_Necesaria,Ganancia_Por_Gol
0,CD Leganés,"$120,000.00","$3,000.00"
1,Valencia CF,"$420,000.00","$7,000.00"
2,RC Celta Vigo,"$230,000.00","$6,000.00"
3,Girona FC,"$100,000.00","$4,000.00"
4,Sevilla FC,"$450,000.00","$10,000.00"


### Filtro según requerimiento los equipos a los cuales puedo llegar con mi inversión de 400.000

In [9]:
df_investment['Inversion_Necesaria'] = df_investment['Inversion_Necesaria'].str.replace('$', '')
df_investment['Inversion_Necesaria'] = df_investment['Inversion_Necesaria'].str.replace(',', '')

df_investment['Inversion_Necesaria'] = df_investment['Inversion_Necesaria'].astype(float)

df_investment[df_investment['Inversion_Necesaria'] <= 400000].sort_values('Inversion_Necesaria')

,Equipo,Inversion_Necesaria,Ganancia_Por_Gol
3,Girona FC,100000.0,"$4,000.00"
9,Málaga CF,100000.0,"$5,000.00"
14,SD Eibar,110000.0,"$7,000.00"
0,CD Leganés,120000.0,"$3,000.00"
13,UD Las Palmas,180000.0,"$7,000.00"
15,RCD Español,180000.0,"$10,000.00"
8,Levante UD,190000.0,"$4,000.00"
12,Deportivo Alavés,210000.0,"$7,000.00"
7,RCD La Coruña,220000.0,"$5,500.00"
2,RC Celta Vigo,230000.0,"$6,000.00"


### Para entender qué equipo me conviene más para invertir, necesito evaluar 2 factores más además de la inversión mínima: cantidad de goles y posición final en el torneo. Por ende, genero una nueva tabla a partir de la anterior, donde genero el cálculo de la ganancia que puedo obtener según la cantidad de goles a favor, y lo ordeno por posición en el torneo (2017-2018):

In [10]:
df_inv_filtrado = df_investment[df_investment['Inversion_Necesaria'] <= 400000].sort_values('Inversion_Necesaria')
df_inv_filtrado['Equipo'] = df_inv_filtrado['Equipo'].str.strip()

df_equipos_inv = pd.merge(df_equipos, df_inv_filtrado, left_on = 'Equipo', right_on = 'Equipo')

df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].str.replace('$', '')
df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].str.replace(',', '')
df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].astype(float)
df_equipos_inv['Ganancia_Total'] = df_equipos_inv.Ganancia_Por_Gol * df_equipos_inv.Goles_A_Favor - df_equipos_inv.Inversion_Necesaria

df_equipos_inv[['Posicion', 'Equipo', 'Puntos', 'Ganancia_Total']].head(20)

,Posicion,Equipo,Puntos,Ganancia_Total
0,6,Real Betis,60,-140000.0
1,8,Getafe CF,55,33000.0
2,9,Girona FC,51,100000.0
3,10,SD Eibar,51,198000.0
4,11,Real Sociedad,49,228000.0
5,12,RCD Español,49,180000.0
6,13,RC Celta Vigo,49,124000.0
7,14,Deportivo Alavés,47,70000.0
8,15,Levante UD,46,-14000.0
9,16,CD Leganés,43,-18000.0


## Se observa que el equipo mejor posicionado y con exposión Internacional (Europa League) es el Real Betis. Sin embargo, con la inversión mínima necesaria y simulando la ganancia que daría el equipo en base a los goles convertidos en la temporada 2017-2018, iríamos a pérdida por un total de 140.000. El resto de los equipos no poseen exposición Internacional pero más de la mitad dan ganancia.

## La mejor inversión que podría hacerse, basándonos en los datos de la última temporada, sería en Real Sociedad. Dista de 20 puntos de la primer posición de descenso de categoría, por ende no se corre riesgo por ese lado; y la ganancia obtenida sería la mejor de toda la Liga con un total de 228.000.

<img src="Real_Sociedad.png" class="center"/>

In [11]:
df_equipos.to_csv('Tabla_2017_2018.csv', index = False)

### El análisis se extendió hacia el resto de las temporadas, como para entender si Real Sociedad es un equipo "regular" y tiene poca probabilidad de perder la categoría. Se generó un dataset con la información de los últimos 10 años, para tener un tablero histórico, y de cada uno de los campos anteriormente calculados, se obtuvo el promedio (de las temporadas disputadas). Además se agrega una columna "Cantidad_Temporadas" donde por cada equipo se determina la cantidad de ligas que disputó en los últimos 10 años:

In [12]:
df_liga_1 = pd.read_csv('2008-09/1-liga.csv')
df_liga_2 = pd.read_csv('2009-10/1-liga.csv')
df_liga_3 = pd.read_csv('2010-11/1-liga.csv')
df_liga_4 = pd.read_csv('2011-12/1-liga.csv')
df_liga_5 = pd.read_csv('2012-13/1-liga.csv')
df_liga_6 = pd.read_csv('2013-14/1-liga.csv')
df_liga_7 = pd.read_csv('2014-15/1-liga.csv')
df_liga_8 = pd.read_csv('2015-16/1-liga.csv')
df_liga_9 = pd.read_csv('2016-17/1-liga.csv')
df_liga_10 = pd.read_csv('2017-18/1-liga.csv')

frames = [df_liga_1, df_liga_2, df_liga_3, df_liga_4, df_liga_5, df_liga_6, df_liga_7, df_liga_8, df_liga_9, df_liga_10]

df_liga_total = pd.concat(frames, ignore_index=True)

df_liga_total.columns = ['Jornada', 'Fecha', 'Local', 'Resultado_Final', 'Resultado_Parcial', 'Visita']

for index, row in df_liga_total.iterrows():
    local = row['Local']
    visita = row['Visita']

    if local.find('(') >= 0:
        local_limpio = local[:local.find('(')].strip()
    else:
        local_limpio = local

    if visita.find('(') >= 0:
        visita_limpio = visita[:visita.find('(')].strip()
    else:
        visita_limpio = visita

    df_liga_total.at[index, 'Local'] = local_limpio
    df_liga_total.at[index, 'Visita'] = visita_limpio

for index, row in df_liga_total.iterrows():
    resultado = row['Resultado_Final']
    local_puntos = 0
    visita_puntos = 0
    
    local_goles = int(resultado[:resultado.find('-')])
    visita_goles = int(resultado[resultado.find('-') + 1:])
    
    if local_goles > visita_goles:
        local_puntos = 3
    else:
        if visita_goles > local_goles:
            visita_puntos = 3
        else:
            local_puntos = 1
            visita_puntos = 1

    df_liga_total.at[index, 'Local_Puntos'] = local_puntos
    df_liga_total.at[index, 'Visita_Puntos'] = visita_puntos
    df_liga_total.at[index, 'Local_Goles'] = local_goles
    df_liga_total.at[index, 'Visita_Goles'] = visita_goles

df_liga_total['Local_Puntos'] = df_liga_total['Local_Puntos'].astype(int)
df_liga_total['Visita_Puntos'] = df_liga_total['Visita_Puntos'].astype(int)
df_liga_total['Local_Goles'] = df_liga_total['Local_Goles'].astype(int)
df_liga_total['Visita_Goles'] = df_liga_total['Visita_Goles'].astype(int)

df_equipos = df_liga_total.groupby('Local').count().reset_index()[['Local']]
df_equipos.columns = ['Equipo']

for index, row in df_equipos.iterrows():
    equipo = row['Equipo']
    puntos = 0
    goles_favor = 0
    goles_contra = 0
    ganados = 0
    empatados = 0
    perdidos = 0
    cant_partidos = 0
    
    df_locales = df_liga_total[df_liga_total['Local'] == equipo]
    df_visitas = df_liga_total[df_liga_total['Visita'] == equipo]
    
    for _, row in df_locales.iterrows():
        local_puntos = row['Local_Puntos']
        local_goles = row['Local_Goles']
        visita_goles = row['Visita_Goles']
        
        puntos += local_puntos
        goles_favor += local_goles
        goles_contra += visita_goles
        cant_partidos += 1
        
        if local_puntos == 3:
            ganados += 1
        else:
            if local_puntos == 1:
                empatados += 1
            else:
                perdidos += 1

    for _, row in df_visitas.iterrows():
        visita_puntos = row['Visita_Puntos']
        visita_goles = row['Visita_Goles']
        local_goles = row['Local_Goles']
        
        puntos += visita_puntos
        goles_favor += visita_goles
        goles_contra += local_goles
        cant_partidos += 1
        
        if visita_puntos == 3:
            ganados += 1
        else:
            if visita_puntos == 1:
                empatados += 1
            else:
                perdidos += 1

    df_equipos.at[index, 'Puntos_P'] = puntos / (cant_partidos / 38)
    df_equipos.at[index, 'Ganados_P'] = ganados / (cant_partidos / 38)
    df_equipos.at[index, 'Empatados_P'] = empatados / (cant_partidos / 38)
    df_equipos.at[index, 'Perdidos_P'] = perdidos / (cant_partidos / 38)
    df_equipos.at[index, 'Goles_A_Favor_P'] = goles_favor / (cant_partidos / 38)
    df_equipos.at[index, 'Goles_En_Contra_P'] = goles_contra / (cant_partidos / 38)
    df_equipos.at[index, 'Diferencia_De_Gol_P'] = (goles_favor - goles_contra) / (cant_partidos / 38)
    df_equipos.at[index, 'Cantidad_Temporadas'] = cant_partidos / 38

df_equipos['Cantidad_Temporadas'] = df_equipos['Cantidad_Temporadas'].astype(int)

df_equipos = df_equipos.sort_values('Puntos_P', ascending = False).reset_index()
df_equipos['Posicion_P'] = df_equipos.index + 1
df_equipos = df_equipos[['Posicion_P', 'Equipo', 'Puntos_P', 'Ganados_P', 'Empatados_P', 'Perdidos_P', 'Goles_A_Favor_P', 'Goles_En_Contra_P', 'Diferencia_De_Gol_P', 'Cantidad_Temporadas']]

#df_equipos[['Posicion_P', 'Equipo', 'Puntos_P', 'Goles_A_Favor_P', 'Cantidad_Temporadas']].head(50)

df_equipos_inv = pd.merge(df_equipos, df_inv_filtrado, left_on = 'Equipo', right_on = 'Equipo')

df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].str.replace('$', '')
df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].str.replace(',', '')
df_equipos_inv['Ganancia_Por_Gol'] = df_equipos_inv['Ganancia_Por_Gol'].astype(float)
df_equipos_inv['Ganancia_Total_P'] = df_equipos_inv.Ganancia_Por_Gol * df_equipos_inv.Goles_A_Favor_P - df_equipos_inv.Inversion_Necesaria

df_equipos_inv[['Posicion_P', 'Equipo', 'Puntos_P', 'Cantidad_Temporadas', 'Ganancia_Total_P']].head(20)

,Posicion_P,Equipo,Puntos_P,Cantidad_Temporadas,Ganancia_Total_P
0,7,Athletic Club Bilbao,54.300000,10,-110050.000000
1,8,Real Sociedad,53.000000,8,141000.000000
2,9,Deportivo Alavés,51.000000,2,73500.000000
3,10,Girona FC,51.000000,1,100000.000000
4,12,RC Celta Vigo,48.500000,6,66000.000000
5,13,RCD Español,46.900000,10,243000.000000
6,14,Málaga CF,46.200000,10,125000.000000
7,15,SD Eibar,45.750000,4,210250.000000
8,16,Getafe CF,45.333333,9,39500.000000
9,18,Real Betis,44.857143,7,-180285.714286


## Observando lo que pasó en los últimos 10 años, se ve cómo Real Sociedad sigue siendo un excelente candidado, con una ganancia promedio de 141.000. Sin embargo, existieron 2 temporadas donde no jugó en primera división, por ende hay un leve riesgo de pérdida de categoría; sumado a eso, aparecen 1 nuevo candidato: RCD Español.

## El RCD Español jugó las últimas 10 termpoadas sin interrupción alguna, y posee un promedio de goles a favor mayor a Real Sociedad, dando una ganancia neta promedio de 243.000, ¡casi el doble que Real Sociedad!

## Del resultado de los últimos 10 años, se considera que la mejor inversión sería en RCD Español. Si se tiene en cuenta la última temporada, Real Sociedad es un equipo superior. Por ende la evaluación final se podría hacer una vez que los planteles decidan refuerzos, bajas y si los equipos técnicos se renuevan o permenecen los mismos de la última temporada.

## Recordemos que los resultados de los partidos de fútbol no son sólo estádistica, sino que hay que tener en cuenta factores anímicos y si los planteles/cuerpos técnicos se mantienen con vigencia. Alguien una vez dijo que el fútbol es "la dinámica de lo impensado"...

<div style="display:flex">
    <div style="flex: 50%; padding: 5px">
        <img src="Real_Sociedad.png" style="float: left;width:400px;height:400px"/>
    </div>
    <div style="flex: 50%; padding: 5px">
        <img src="RCD_Espanyol.png" style="float: left;width:300px;height300px"/>
    </div>
</div>